<a href="https://colab.research.google.com/github/jdariasl/ML_2020/blob/master/Labs/Extra/DespligueModelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Despliegue de modelos

## Introducción

Recordar los pasos que habíamos visto en los estándares introducidos en [nuestra sesión](Basic_Preprocessing_FeatureEngineering.ipynb):

1. Entender problema y el objetivo
2. Obtener y Entender los datos (análisis exploratorio-- en algunas prácticas hemos iniciado con estas técnicas)
3. Modelar: incluyendo las transformaciones/ limpieza/ ingeniería de características de los datos y el modelo de ML (entrenamiento, validación, experimentos -- lo que hemos estado haciendo en las practicas)
4. ** Despliegue del Modelo**









### La Practica de la Ciencia de Datos

A la hora de hablar de despliegue de modelos, nos encontramos con una discusión de roles que muchas veces se superponen. Esta superposición Se ve influenciada por diversos factores externos, como el tamaño de la compañía, el equipo y contexto particular.

<img src="https://raw.githubusercontent.com/jdariasl/ML_2020/master/Labs/Extra/UDEA_DS_ML.jpg" alt="drawing" width="800"/>


## Formas de Despliegue

En la practica hay varios escenarios para realizar la puesta en marcha de los modelos. Como es usual depende del contexto y necesidades particulares.

### Batch o por lotes


<img src="https://raw.githubusercontent.com/jdariasl/ML_2020/master/Labs/Extra/batch.jpg" alt="drawing" width="800"/>


**Ejemplo**

Cada Trimestre una compañía de crédito, actualiza la información de sus usuarios, con esa información en la compañía predice la capacidad de pago de sus clientes. Esa predicción es almacenada y es consumida por diferentes aplicaciones y sistemas de la compañía.







### Tiempo real usando Feature Lookup

<img src="https://raw.githubusercontent.com/jdariasl/ML_2020/master/Labs/Extra/feature_lookup.jpg" alt="drawing" width="800"/>



**Ejemplo**

Una compañía de pagos creo un modelo de predicción de fraude. 
El modelo necesita 100 Variables. 
Las 100 Variables requieren de agregaciones a nivel de usuario y otros factores. 

En este caso el servicio que consume el modelo de predicción de fraude solo envía unos identificadores clave como el usuario y la tarjeta. 

El modelo recibe esa información y realizar la consulta a una base de datos de cache para derivar las características. Luego de obtener las características, aplica el modelo y responde con la probabilidad de fraude.


### Tiempo real

<img src="https://raw.githubusercontent.com/jdariasl/ML_2020/master/Labs/Extra/realtime.jpg" alt="drawing" width="800"/>



**Ejemplo**

[Azure Vision](https://azure.microsoft.com/es-es/services/cognitive-services/computer-vision/#features)





## Entrenamiento del modelo

In [5]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris



X,y = load_iris(return_X_y = True)

Sklearn maneja un concepto par apoder agrupar todos los pasos de procesamiento.

Los Pipelines.


In [ ]:
from sklearn.preprocessing import StandardScaler


scaler = MinMaxScaler()
scaled_X = scaler.fit_transform(X)

## Re-factorización

## Despliegue a servicio web

## Rerencias y enlaces de interes

https://jakevdp.github.io/PythonDataScienceHandbook/05.04-feature-engineering.html

https://pandas.pydata.org/pandas-docs/stable/user_guide/missing_data.html

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

https://scikit-learn.org/stable/modules/impute.html